In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import json
from multiprocessing import Pool
from lampe.data import JointLoader
from itertools import islice
from tqdm import tqdm
from lampe.diagnostics import expected_coverage_mc
from lampe.plots import coverage_plot

import sys

sys.path.insert(0, "../scr/cryo_sbi/inference/")
sys.path.insert(0, "../scr/cryo_sbi/inference/models")
sys.path.insert(0, "../scr/cryo_sbi/wpa_simulator/")

from models import build_models
from cryo_em_simulator import CryoEmSimulator
import priors

In [ ]:
file_name = "TEST"  # File name
config_dir = "../experiments/benchmark_hsp90/"
num_samples_stats = 20000  # Number of simulations for computing posterior stats
num_samples_SBC = 10000  # Number of simulations for SBC
num_posterior_samples_SBC = 1024  # Number of posterior samples for each SBC simulation
num_samples_posterior = 50000  # Number of samples to draw from posterior
batch_size_sampling = 100  # Batch size for sampling posterior
num_workers = 24  # Number of CPU cores
device = "cuda"  # Device for computations
save_figures = False

## Load cryo-em simulator and posterior with correct config

In [ ]:
cryosbi = CryoEmSimulator(config_dir + "image_params_snr01_128.json")

In [ ]:
train_config = json.load(open(config_dir + "resnet18_encoder.json"))
estimator = build_models.build_npe_flow_model(train_config)
estimator.load_state_dict(torch.load(config_dir + "resnet18_encoder_snr01.estimator"))
estimator.cuda()
estimator.eval();

## Plott the training loss

## Testing posterior on array of images

In [ ]:
indices = torch.tensor(np.arange(0, cryosbi.max_index + 1, 1), dtype=float).reshape(
    -1, 1
)
images = torch.stack([cryosbi.simulator(index) for index in indices], dim=0)

In [ ]:
fig, axes = plt.subplots(4, 5, figsize=(10, 8))
for idx, ax in enumerate(axes.reshape(-1)):
    ax.imshow(images[idx], vmax=5, vmin=-5)
    ax.set_yticks([])
    ax.set_xticks([])
    ax.text(10, 30, str(int(indices[idx].item())))

if save_figures:
    fig.savefig(f"{data_dir}particles_6wxb_test", dpi=500)

In [ ]:
import torchvision.transforms as T

transform = T.Resize(size=(128, 128))

In [ ]:
import torchvision.transforms as T

transform = T.Resize(size=(128, 128))


def gaussian_normalize_image(image):
    image = transform(image)
    mean_img = torch.mean(image)
    std_img = torch.std(image)

    return (image - mean_img) / std_img

In [ ]:
for idx, image in enumerate(images):
    print(images[idx].shape)

In [ ]:
plt.imshow(images[0])

In [ ]:
theta_samples = []
with torch.no_grad():
    for batched_images in torch.split(
        images, split_size_or_sections=batch_size_sampling, dim=0
    ):
        samples = estimator.sample(
            gaussian_normalize_image(batched_images.cuda(non_blocking=True)),
            shape=(num_samples_posterior,),
        ).cpu()
        theta_samples.append(samples.reshape(-1, batch_size_sampling))
    samples = torch.cat(theta_samples, dim=1)

In [ ]:
fig, axes = plt.subplots(4, 5, figsize=(10, 8), sharex=True)
for idx, ax in enumerate(axes.reshape(1, -1)[0]):
    ax.hist(
        samples[:, idx].flatten().numpy(),
        bins=np.arange(0, 20, 0.5),
        histtype="step",
        color="blue",
        label="all",
    )
    ax.set_yticks([])
    ax.set_yticks([])
    ax.set_xticks(range(0, 20, 4))
    ax.axvline(indices[idx], color="red")
# plt.savefig('posterior_snr001_widres50_128x128.pdf', dpi=500)

## Compute posterior calibration

In [ ]:
loader = JointLoader(
    priors.get_unirom_prior_1d(cryosbi.get_max_index()),
    cryosbi.simulator,
    vectorized=False,
    batch_size=1,
    num_workers=num_workers,
    prefetch_factor=1,
)

levels, coverages = expected_coverage_mc(
    estimator.flow,
    (
        (estimator.standardize(theta.cuda()), gaussian_normalize_image(x.cuda()))
        for theta, x in islice(loader, 5000)
    ),
    n=num_posterior_samples_SBC,
)

fig = coverage_plot(levels, coverages, legend="NPE")
fig.savefig(f"SBC_normal.pdf", dpi=500)
if save_figures:
    fig.savefig(f"{data_dir}sbc_{file_name}.pdf", dpi=500)

## Compute posterior varaicen as function of projection angle

In [ ]:
quats = torch.from_numpy(np.load("quaternion_list.npy"))
num_simulations = len(quats)
indices = 10 * torch.ones(num_simulations).reshape(-1, 1)
parameters = torch.cat((indices, quats), dim=1)
images = torch.stack(
    [cryosbi._simulator_with_quat(param) for param in parameters], dim=0
)

In [ ]:
theta_samples = []
with torch.no_grad():
    for batched_images in torch.split(
        images, split_size_or_sections=batch_size_sampling, dim=0
    ):
        samples = estimator.sample(
            batched_images.cuda(non_blocking=True), shape=(num_samples_posterior,)
        ).cpu()
        theta_samples.append(samples.reshape(-1, batch_size_sampling))
samples = torch.cat(theta_samples, dim=1)

In [ ]:
posterior_quantiles = np.quantile(samples.numpy(), [0.025, 0.975], axis=0)
confidence_widths = posterior_quantiles[1] - posterior_quantiles[0]

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import cm, colors
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from scipy.spatial.transform import Rotation

# Create a sphere
r = 1
pi = np.pi
cos = np.cos
sin = np.sin
phi, theta = np.mgrid[0.0:pi:100j, 0.0 : 2.0 * pi : 100j]
x = r * sin(phi) * cos(theta)
y = r * sin(phi) * sin(theta)
z = r * cos(phi)

unit_vecotr = np.array([0, 0, 1])
points = []
for i in range(N_samples):
    rot_mat = Rotation.from_quat(quats[i]).as_matrix()
    coord = np.matmul(rot_mat, unit_vecotr)
    points.append(coord)
points = np.array(points)

xx = points[:, 0]
yy = points[:, 1]
zz = points[:, 2]

# Set colours and render
fig = plt.figure()
ax = fig.add_subplot(111, projection="3d")

# ax.plot_surface(
#    x, y, z,  rstride=1, cstride=1, color='c', alpha=0.3, linewidth=0)

im = ax.scatter(xx, yy, zz, s=confidence_widths, c=confidence_widths)

ax.set_xlabel("x")
ax.set_ylabel("y")
# ax.set_zlabel('z')
ax.set_xticklabels([])
ax.set_yticklabels([])
ax.set_zticklabels([])

ax.azim = -40
ax.elev = 30
ax.dist = 10

fig.colorbar(im)

ax.set_xlim([-1, 1])
ax.set_ylim([-1, 1])
ax.set_zlim([-1, 1])
ax.set_aspect("auto")
plt.tight_layout()
plt.show()
# plt.savefig("rotation_posterior_widths_snr001_widres50_128x128.pdf", dpi=600)

In [ ]:
def asSpherical(xyz):
    x = xyz[0]
    y = xyz[1]
    z = xyz[2]
    r = np.sqrt(x * x + y * y + z * z)
    theta = np.arccos(z / r)
    phi = np.arctan2(y, x)
    return [r, theta, phi]


coordinates = np.array([asSpherical(point)[1:] for point in points])

In [ ]:
plt.scatter(
    coordinates[:, 0], coordinates[:, 1], s=confidence_widths, c=confidence_widths
)
plt.colorbar(label=r"width of 3$\sigma$-confidence-intervall")
plt.xlabel(r"$\theta$", fontsize=14)
plt.ylabel(r"$\phi$", fontsize=14)
plt.xticks(
    ticks=[0, np.pi / 4, np.pi / 2, 3 * np.pi / 4, np.pi],
    labels=[
        "0",
        r"$\frac{1}{4}\pi$",
        r"$\frac{1}{2}\pi$",
        r"$\frac{3}{4}\pi$",
        r"$\pi$",
    ],
)
plt.yticks(
    ticks=[-np.pi, -np.pi / 2, 0, np.pi / 2, np.pi],
    labels=[r"$-\pi$", r"$-\frac{1}{2}\pi$", r"0", r"$\frac{1}{2}\pi$", r"$\pi$"],
)
plt.title("Index 10")
plt.savefig(f"Rotations_{file_name}.pdf", dpi=500)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import cm, colors
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

# Set colours and render
fig = plt.figure()
ax = fig.add_subplot(111, projection="3d")

x, y, z = cryosbi.models[0]
ax.scatter(x, y, z, s=20)

ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("z")

ax.azim = 230
ax.elev = 40
ax.dist = 10

# ax.set_xlim([-1,1])
# ax.set_ylim([-1,1])
# ax.set_zlim([-1,1])
ax.set_aspect("auto")
plt.tight_layout()
# plt.savefig("inital_rotation_posterior_widths_snr01_widres50_128x128.pdf", dpi=600)
plt.show()